### 1. Blackjack with Monte Carlo Method

#### Theory:
Blackjack is a popular card game where the objective is to accumulate a hand value as close to 21 as possible without exceeding it. The game involves probabilistic decisions, making it a suitable candidate for reinforcement learning techniques like Monte Carlo methods.

**Monte Carlo methods** involve using randomness to solve problems that might be deterministic in principle. In the context of Blackjack, Monte Carlo methods are used to evaluate and improve the strategies by simulating a large number of games. The key idea is to average the returns (rewards) following visits to a particular state to estimate the value function, which is the expected return starting from that state.

**Steps:**
1. **Simulate Episodes:** Play the game repeatedly by following a certain policy.
2. **Track Returns:** For each state encountered during the episodes, track the rewards received.
3. **Update Value Function:** Average the returns to update the value function for each state.


In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def run_episode(env, hold_score):
    """
    Run a single episode of Blackjack until the game is done.

    Parameters:
    env (gym.Env): The Blackjack environment.
    hold_score (int): The score at which the player will hold (stop taking cards).

    Returns:
    states (list): List of states encountered during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    states = [state]
    is_done = False

    while not is_done:
        # Take action: hit (1) if score < hold_score, else stick (0)
        action = 1 if state[0] < hold_score else 0
        state, reward, is_done, _ = env.step(action)
        states.append(state)
        rewards.append(reward)

    return states, rewards

In [ ]:
def mc_prediction_first_visit(env, hold_score, gamma, n_episode):
    """
    Monte Carlo prediction algorithm to estimate the value function.

    Parameters:
    env (gym.Env): The Blackjack environment.
    hold_score (int): The score at which the player will hold.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.

    Returns:
    V (defaultdict): The estimated value function.
    """
    V = defaultdict(float)
    N = defaultdict(int)

    for episode in range(n_episode):
        states_t, rewards_t = run_episode(env, hold_score)
        return_t = 0
        G = {}

        for state_t, reward_t in zip(states_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            if state_t not in G:
                G[state_t] = return_t

        for state, return_t in G.items():
            if state[0] <= 21:  # Consider only valid states
                V[state] += return_t
                N[state] += 1

    for state in V:
        V[state] /= N[state]

    return V

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
hold_score = 18
gamma = 1
n_episode = 500000

# Run Monte Carlo prediction
value = mc_prediction_first_visit(env, hold_score, gamma, n_episode)

# Print the value function
print('Value function estimated using First-Visit MC:\n', value)
print('Number of states:', len(value))

# Plot the value function
plot_blackjack_value(value)

### 2. Monte Carlo Control with On-Policy Strategy

#### Theory:
**On-policy Monte Carlo control** aims to find the optimal policy while using and improving the policy being followed. The policy is improved iteratively based on the value function derived from the returns of the simulated episodes.

**Steps:**
1. **Initialize Policy and Q-Function:** Start with an initial policy and Q-function.
2. **Simulate Episodes:** Follow the current policy to generate episodes.
3. **Update Q-Function:** Use the returns from the episodes to update the Q-function.
4. **Improve Policy:** Use the updated Q-function to improve the policy by making it greedy with respect to the Q-function.

In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def run_episode(env, Q, n_action):
    """
    Executes an episode following the given Q-function.

    Parameters:
    env (gym.Env): The Blackjack environment.
    Q (defaultdict): The Q-function.
    n_action (int): The number of possible actions.

    Returns:
    states (list): List of states encountered during the episode.
    actions (list): List of actions taken during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    actions = []
    states = []
    is_done = False

    # Start with a random action
    action = torch.randint(0, n_action, [1]).item()

    while not is_done:
        actions.append(action)
        states.append(state)
        state, reward, is_done, _ = env.step(action)
        rewards.append(reward)

        if is_done:
            break

        # Choose the next action based on the current Q-function
        action = torch.argmax(Q[state]).item()

    return states, actions, rewards

In [ ]:
def mc_control_on_policy(env, gamma, n_episode):
    """
    Finds the optimal policy using on-policy Monte Carlo control.

    Parameters:
    env (gym.Env): The Blackjack environment.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.

    Returns:
    Q (defaultdict): The optimal Q-function.
    policy (dict): The optimal policy.
    """
    n_action = env.action_space.n
    G_sum = defaultdict(float)
    N = defaultdict(int)
    Q = defaultdict(lambda: torch.zeros(n_action))

    for episode in range(n_episode):
        states_t, actions_t, rewards_t = run_episode(env, Q, n_action)
        return_t = 0
        G = {}

        # Calculate the return for each state-action pair
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            if (state_t, action_t) not in G:
                G[(state_t, action_t)] = return_t

        # Update Q-function based on the returns
        for state_action, return_t in G.items():
            state, action = state_action
            if state[0] <= 21:  # Consider only valid states
                G_sum[state_action] += return_t
                N[state_action] += 1
                Q[state][action] = G_sum[state_action] / N[state_action]

    # Derive the optimal policy from the Q-function
    policy = {state: torch.argmax(actions).item() for state, actions in Q.items()}

    return Q, policy

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
gamma = 1
n_episode = 500000

# Run Monte Carlo control to find the optimal policy and Q-function
optimal_Q, optimal_policy = mc_control_on_policy(env, gamma, n_episode)

# Print the optimal policy
print("Optimal Policy:\n", optimal_policy)

# Derive the value function from the optimal Q-function
optimal_value = defaultdict(float)
for state, action_values in optimal_Q.items():
    optimal_value[state] = torch.max(action_values).item()

# Print the optimal value function
print("Optimal Value Function:\n", optimal_value)

# Plot the optimal value function
plot_blackjack_value(optimal_value)

### 3. Monte Carlo Method with ε-Greedy Strategy

#### Theory:
The **ε-greedy strategy** is used to balance exploration and exploitation in reinforcement learning. It ensures that the agent occasionally explores new actions to discover their potential benefits while mostly exploiting known actions that yield the highest rewards.

**Steps:**
1. **Initialize Q-Function:** Start with an initial Q-function.
2. **Simulate Episodes:** Use the ε-greedy policy to generate episodes, where the agent chooses a random action with probability ε and the best-known action with probability 1-ε.
3. **Update Q-Function:** Use the returns from the episodes to update the Q-function.
4. **Improve Policy:** The policy becomes greedy with respect to the updated Q-function.

In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def run_episode(env, Q, epsilon, n_action):
    """
    Executes an episode following the ε-greedy strategy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    Q (defaultdict): The Q-function.
    epsilon (float): The exploration-exploitation trade-off parameter.
    n_action (int): The number of possible actions.

    Returns:
    states (list): List of states encountered during the episode.
    actions (list): List of actions taken during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    actions = []
    states = []
    is_done = False

    while not is_done:
        # Epsilon-greedy action selection
        probs = torch.ones(n_action) * epsilon / n_action
        best_action = torch.argmax(Q[state]).item()
        probs[best_action] += 1.0 - epsilon
        action = torch.multinomial(probs, 1).item()

        actions.append(action)
        states.append(state)
        state, reward, is_done, _ = env.step(action)
        rewards.append(reward)

    return states, actions, rewards

In [ ]:
def mc_control_epsilon_greedy(env, gamma, n_episode, epsilon):
    """
    Finds the optimal ε-greedy policy using Monte Carlo control.

    Parameters:
    env (gym.Env): The Blackjack environment.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.
    epsilon (float): The exploration-exploitation trade-off parameter.

    Returns:
    Q (defaultdict): The optimal Q-function.
    policy (dict): The optimal policy.
    """
    n_action = env.action_space.n
    G_sum = defaultdict(float)
    N = defaultdict(int)
    Q = defaultdict(lambda: torch.zeros(n_action))

    for episode in range(n_episode):
        states_t, actions_t, rewards_t = run_episode(env, Q, epsilon, n_action)
        return_t = 0
        G = {}

        # Calculate the return for each state-action pair
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            if (state_t, action_t) not in G:
                G[(state_t, action_t)] = return_t

        # Update Q-function based on the returns
        for state_action, return_t in G.items():
            state, action = state_action
            if state[0] <= 21:  # Consider only valid states
                G_sum[state_action] += return_t
                N[state_action] += 1
                Q[state][action] = G_sum[state_action] / N[state_action]

    # Derive the optimal policy from the Q-function
    policy = {state: torch.argmax(actions).item() for state, actions in Q.items()}

    return Q, policy

In [ ]:
def simulate_episode(env, policy):
    """
    Simulates an episode using a given policy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    policy (dict): The policy to follow.

    Returns:
    reward (int): The reward received at the end of the episode.
    """
    state = env.reset()
    is_done = False

    while not is_done:
        action = policy[state]
        state, reward, is_done, _ = env.step(action)

    return reward

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
gamma = 1
n_episode = 500000
epsilon = 0.1

# Run Monte Carlo control to find the optimal ε-greedy policy and Q-function
optimal_Q, optimal_policy = mc_control_epsilon_greedy(env, gamma, n_episode, epsilon)

# Print the optimal policy
print("Optimal Policy:\n", optimal_policy)

# Derive the value function from the optimal Q-function
optimal_value = defaultdict(float)
for state, action_values in optimal_Q.items():
    optimal_value[state] = torch.max(action_values).item()

# Print the optimal value function
print("Optimal Value Function:\n", optimal_value)

# Plot the optimal value function
plot_blackjack_value(optimal_value)

In [ ]:
# Simulate episodes to evaluate the optimal policy
n_episode = 100000
n_win_optimal = 0
n_lose_optimal = 0

for _ in range(n_episode):
    reward = simulate_episode(env, optimal_policy)
    if reward == 1:
        n_win_optimal += 1
    elif reward == -1:
        n_lose_optimal += 1

# Print the win/lose probabilities
print('Win probability with optimal policy: {:.2f}'.format(n_win_optimal / n_episode))
print('Lose probability with optimal policy: {:.2f}'.format(n_lose_optimal / n_episode))

### 4. Monte Carlo Control with Off-Policy Strategy

#### Theory:
**Off-policy Monte Carlo control** involves learning an optimal policy while following a different behavior policy. This approach uses importance sampling to correct the difference between the behavior policy and the target policy.

**Steps:**
1. **Behavior Policy:** Define a behavior policy that is used to generate episodes.
2. **Simulate Episodes:** Follow the behavior policy to generate episodes.
3. **Importance Sampling:** Use importance sampling to weight the returns, correcting for the difference between the behavior policy and the target policy.
4. **Update Q-Function:** Use the weighted returns to update the Q-function.
5. **Improve Policy:** The policy is improved based on the updated Q-function.

In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def gen_random_policy(n_action):
    """
    Generates a random policy with equal probability for each action.

    Parameters:
    n_action (int): Number of possible actions.

    Returns:
    function: A policy function that returns action probabilities for a given state.
    """
    probs = torch.ones(n_action) / n_action
    def policy_function(state):
        return probs
    return policy_function

In [ ]:
random_policy = gen_random_policy(env.action_space.n)

In [ ]:
def run_episode(env, behavior_policy):
    """
    Executes an episode following the given behavior policy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    behavior_policy (function): The behavior policy function.

    Returns:
    states (list): List of states encountered during the episode.
    actions (list): List of actions taken during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    actions = []
    states = []
    is_done = False

    while not is_done:
        probs = behavior_policy(state)
        action = torch.multinomial(probs, 1).item()
        actions.append(action)
        states.append(state)
        state, reward, is_done, _ = env.step(action)
        rewards.append(reward)

    return states, actions, rewards

In [ ]:
def mc_control_off_policy(env, gamma, n_episode, behavior_policy):
    """
    Finds the optimal policy using off-policy Monte Carlo control with importance sampling.

    Parameters:
    env (gym.Env): The Blackjack environment.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.
    behavior_policy (function): The behavior policy function.

    Returns:
    Q (defaultdict): The optimal Q-function.
    policy (dict): The optimal policy.
    """
    n_action = env.action_space.n
    G_sum = defaultdict(float)
    N = defaultdict(int)
    Q = defaultdict(lambda: torch.zeros(n_action))

    for episode in range(n_episode):
        states_t, actions_t, rewards_t = run_episode(env, behavior_policy)
        return_t = 0
        G = {}
        w = 1  # Importance sampling ratio

        # Calculate the return for each state-action pair
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            G[(state_t, action_t)] = return_t

            if action_t != torch.argmax(Q[state_t]).item():
                break
            w *= 1.0 / behavior_policy(state_t)[action_t]

        # Update Q-function based on the returns
        for state_action, return_t in G.items():
            state, action = state_action
            if state[0] <= 21:  # Consider only valid states
                G_sum[state_action] += return_t * w
                N[state_action] += 1
                Q[state][action] = G_sum[state_action] / N[state_action]

    # Derive the optimal policy from the Q-function
    policy = {state: torch.argmax(actions).item() for state, actions in Q.items()}

    return Q, policy

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
gamma = 1
n_episode = 500000

# Run Monte Carlo control to find the optimal policy and Q-function
optimal_Q, optimal_policy = mc_control_off_policy(env, gamma, n_episode, random_policy)

# Print the optimal policy
print("Optimal Policy:\n", optimal_policy)

# Derive the value function from the optimal Q-function
optimal_value = defaultdict(float)
for state, action_values in optimal_Q.items():
    optimal_value[state] = torch.max(action_values).item()

# Print the optimal value function
print("Optimal Value Function:\n", optimal_value)

# Plot the optimal value function
plot_blackjack_value(optimal_value)

### 5. Incremental Implementation of the Previous Method


#### Theory:
**Incremental Monte Carlo control** updates the Q-function incrementally after each episode, rather than waiting for a batch of episodes. This approach is computationally efficient and helps in real-time learning scenarios.

**Steps:**
1. **Initialize Q-Function and Count:** Start with initial Q-values and counts for state-action pairs.
2. **Simulate Episodes:** Use a behavior policy to generate episodes.
3. **Update Incrementally:** For each state-action pair in the episode, update the Q-function incrementally using the observed return and importance sampling weights.
4. **Improve Policy:** Update the policy to be greedy with respect to the incrementally updated Q-function.

In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def gen_random_policy(n_action):
    """
    Generates a random policy with equal probability for each action.

    Parameters:
    n_action (int): Number of possible actions.

    Returns:
    function: A policy function that returns action probabilities for a given state.
    """
    probs = torch.ones(n_action) / n_action
    def policy_function(state):
        return probs
    return policy_function

In [ ]:
random_policy = gen_random_policy(env.action_space.n)

In [ ]:
def run_episode(env, behavior_policy):
    """
    Executes an episode following the given behavior policy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    behavior_policy (function): The behavior policy function.

    Returns:
    states (list): List of states encountered during the episode.
    actions (list): List of actions taken during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    actions = []
    states = []
    is_done = False

    while not is_done:
        probs = behavior_policy(state)
        action = torch.multinomial(probs, 1).item()
        actions.append(action)
        states.append(state)
        state, reward, is_done, _ = env.step(action)
        rewards.append(reward)

    return states, actions, rewards

In [ ]:
def mc_control_off_policy_incremental(env, gamma, n_episode, behavior_policy):
    """
    Finds the optimal policy using off-policy Monte Carlo control with incremental updates.

    Parameters:
    env (gym.Env): The Blackjack environment.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.
    behavior_policy (function): The behavior policy function.

    Returns:
    Q (defaultdict): The optimal Q-function.
    policy (dict): The optimal policy.
    """
    n_action = env.action_space.n
    N = defaultdict(int)
    Q = defaultdict(lambda: torch.zeros(n_action))

    for episode in range(n_episode):
        W = 1.0  # Importance sampling ratio
        states_t, actions_t, rewards_t = run_episode(env, behavior_policy)
        return_t = 0.0

        # Calculate the return for each state-action pair
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            N[(state_t, action_t)] += 1
            Q[state_t][action_t] += (W / N[(state_t, action_t)]) * (return_t - Q[state_t][action_t])

            if action_t != torch.argmax(Q[state_t]).item():
                break
            W *= 1.0 / behavior_policy(state_t)[action_t]

    # Derive the optimal policy from the Q-function
    policy = {state: torch.argmax(actions).item() for state, actions in Q.items()}

    return Q, policy

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
gamma = 1
n_episode = 500000

# Run Monte Carlo control to find the optimal policy and Q-function
optimal_Q, optimal_policy = mc_control_off_policy_incremental(env, gamma, n_episode, random_policy)

# Print the optimal policy
print("Optimal Policy:\n", optimal_policy)

# Derive the value function from the optimal Q-function
optimal_value = defaultdict(float)
for state, action_values in optimal_Q.items():
    optimal_value[state] = torch.max(action_values).item()

# Print the optimal value function
print("Optimal Value Function:\n", optimal_value)

# Plot the optimal value function
plot_blackjack_value(optimal_value)

### 6. Monte Carlo Control with Weighted Importance Sampling

#### Theory:
**Weighted importance sampling** is an advanced method in off-policy Monte Carlo control. It adjusts the updates to the Q-function using the cumulative importance sampling ratios, ensuring that the estimates remain unbiased and efficient.

**Steps:**
1. **Behavior Policy:** Define a behavior policy for generating episodes.
2. **Simulate Episodes:** Generate episodes following the behavior policy.
3. **Importance Sampling:** Calculate the cumulative importance sampling ratios to weight the returns appropriately.
4. **Update Q-Function:** Update the Q-function using weighted returns to reflect the true value under the target policy.
5. **Improve Policy:** Derive the optimal policy based on the updated Q-function.


In [ ]:
import torch
import gym
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Initialize the Blackjack environment
env = gym.make('Blackjack-v1')

In [ ]:
def gen_random_policy(n_action):
    """
    Generates a random policy with equal probability for each action.

    Parameters:
    n_action (int): Number of possible actions.

    Returns:
    function: A policy function that returns action probabilities for a given state.
    """
    probs = torch.ones(n_action) / n_action

    def policy_function(state):
        return probs

    return policy_function

In [ ]:
random_policy = gen_random_policy(env.action_space.n)

In [ ]:
def run_episode(env, behavior_policy):
    """
    Executes an episode following the given behavior policy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    behavior_policy (function): The behavior policy function.

    Returns:
    states (list): List of states encountered during the episode.
    actions (list): List of actions taken during the episode.
    rewards (list): List of rewards received during the episode.
    """
    state = env.reset()
    rewards = []
    actions = []
    states = []
    is_done = False

    while not is_done:
        probs = behavior_policy(state)
        action = torch.multinomial(probs, 1).item()
        actions.append(action)
        states.append(state)
        state, reward, is_done, _ = env.step(action)
        rewards.append(reward)

    return states, actions, rewards

In [ ]:
def mc_control_off_policy_weighted(env, gamma, n_episode, behavior_policy):
    """
    Finds the optimal policy using off-policy Monte Carlo control with weighted importance sampling.

    Parameters:
    env (gym.Env): The Blackjack environment.
    gamma (float): Discount factor.
    n_episode (int): Number of episodes to run.
    behavior_policy (function): The behavior policy function.

    Returns:
    Q (defaultdict): The optimal Q-function.
    policy (dict): The optimal policy.
    """
    n_action = env.action_space.n
    N = defaultdict(float)
    Q = defaultdict(lambda: torch.zeros(n_action))

    for episode in range(n_episode):
        W = 1.0  # Importance sampling ratio
        states_t, actions_t, rewards_t = run_episode(env, behavior_policy)
        return_t = 0.0

        # Calculate the return for each state-action pair
        for state_t, action_t, reward_t in zip(states_t[::-1], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            N[(state_t, action_t)] += W
            Q[state_t][action_t] += (W / N[(state_t, action_t)]) * (return_t - Q[state_t][action_t])

            if action_t != torch.argmax(Q[state_t]).item():
                break
            W *= 1.0 / behavior_policy(state_t)[action_t]

    # Derive the optimal policy from the Q-function
    policy = {state: torch.argmax(actions).item() for state, actions in Q.items()}

    return Q, policy

In [ ]:
def simulate_episode(env, policy):
    """
    Simulates an episode using a given policy.

    Parameters:
    env (gym.Env): The Blackjack environment.
    policy (dict): The policy to follow.

    Returns:
    reward (int): The reward received at the end of the episode.
    """
    state = env.reset()
    is_done = False

    while not is_done:
        action = policy[state]
        state, reward, is_done, _ = env.step(action)

    return reward

In [ ]:
def plot_surface(X, Y, Z, title):
    """
    Plot a 3D surface.

    Parameters:
    X (array): Meshgrid array for X-axis.
    Y (array): Meshgrid array for Y-axis.
    Z (array): Values for Z-axis.
    title (str): Plot title.
    """
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=matplotlib.cm.coolwarm, vmin=-1.0, vmax=1.0)
    ax.set_xlabel('Player Sum')
    ax.set_ylabel('Dealer Showing')
    ax.set_zlabel('Value')
    ax.set_title(title)
    ax.view_init(ax.elev, -120)
    fig.colorbar(surf)
    plt.show()

In [ ]:
def plot_blackjack_value(V):
    """
    Plot the value function for Blackjack.

    Parameters:
    V (defaultdict): The estimated value function.
    """
    player_sum_range = range(12, 22)
    dealer_show_range = range(1, 11)
    X, Y = torch.meshgrid([torch.tensor(player_sum_range), torch.tensor(dealer_show_range)])

    values_to_plot = torch.zeros((len(player_sum_range), len(dealer_show_range), 2))

    for i, player in enumerate(player_sum_range):
        for j, dealer in enumerate(dealer_show_range):
            for k, ace in enumerate([False, True]):
                values_to_plot[i, j, k] = V[(player, dealer, ace)]

    plot_surface(X, Y, values_to_plot[:, :, 0].numpy(), "Value Function without Usable Ace")
    plot_surface(X, Y, values_to_plot[:, :, 1].numpy(), "Value Function with Usable Ace")

In [ ]:
# Parameters
gamma = 1
n_episode = 500000

# Run Monte Carlo control to find the optimal policy and Q-function
optimal_Q, optimal_policy = mc_control_off_policy_weighted(env, gamma, n_episode, random_policy)

# Print the optimal policy
print("Optimal Policy:\n", optimal_policy)

# Derive the value function from the optimal Q-function
optimal_value = defaultdict(float)
for state, action_values in optimal_Q.items():
    optimal_value[state] = torch.max(action_values).item()

# Print the optimal value function
print("Optimal Value Function:\n", optimal_value)

# Plot the optimal value function
plot_blackjack_value(optimal_value)

In [ ]:
# Simulate episodes to evaluate the optimal policy
n_episode = 100000
n_win_optimal = 0
n_lose_optimal = 0

for _ in range(n_episode):
    reward = simulate_episode(env, optimal_policy)
    if reward == 1:
        n_win_optimal += 1
    elif reward == -1:
        n_lose_optimal += 1

# Print the win/lose probabilities
print('Win probability with optimal policy: {:.2f}'.format(n_win_optimal / n_episode))
print('Lose probability with optimal policy: {:.2f}'.format(n_lose_optimal / n_episode))